In [ ]:
import pyspark
import random
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
conf = SparkConf().setAppName('Assignment no 3').setMaster('local[*]')
sc = SparkContext(conf=conf)

In [ ]:
# QUestion no 1
spark = SparkSession.builder.appName("Ano3_Qno1").getOrCreate()
Sentences = spark.read.text("InputSentences.txt").rdd.map(lambda r: r[0])

Sentences_ = Sentences.map(lambda word: word.upper())

def threelongconsecutivewords(line):
    words = line.split()
    result = []
    for i in range(len(words) - 2):
        if (len(words[i]) > 4) and (len(words[i+1]) >= 4) and (len(words[i+2]) >= 4) and (words[i][0] == words[i+1][0] == words[i+2][0]):
            result.append(words[i] + " " + words[i+1] + " " + words[i+2])
    return result

long_words = Sentences_.flatMap(threelongconsecutivewords)

word_counts = long_words.map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)
word = word_counts.map(lambda x:x[1]).sum()

alphabet_count = long_words.map(lambda x: (x[0], 1)).reduceByKey(lambda a, b: a + b)
alpha_count =  alphabet_count.map(lambda x: (x[0][0], x[1])).reduceByKey(lambda a, b: a + b)

print("Total count:", word)
print()
for letter, count in alpha_count.collect():
    print(letter, " : ",count)
print()
for word, count in word_counts.collect():
    print(word," : ",count)

Total count: 7

H  =>  3
M  =>  1
S  =>  1
D  =>  1
F  =>  1

HORRID HENRY’S HOUND  =>  2
HENRY’S HOUND HUNTS  =>  1
MASSIVE MURREE MOUNTAINS.  =>  1
SILLY STUPID SAMUEL’S  =>  1
DREADFUL DRAGON DANY  =>  1
FANTASTIC FANCIFUL FOURSOME.  =>  1


In [ ]:
#Question no 2
input_file = sc.textFile("InputFile.txt")
input_data = input_file.map(lambda line: line.split(":"))
input_data = input_data.map(lambda x: (x[0], list(map(int, x[1].split()))))


reference_file = sc.textFile("Reference.txt")
reference_data = reference_file.map(lambda line: line.split(":"))
reference_data = reference_data.map(lambda x: (x[0], list(map(int, x[1].split()))))

# Compute Jaccard distance function
def jaccard_distance(a, b):
    intersection = sum([1 for i in range(len(a)) if a[i] == 1 and b[i] == 1])
    union = sum([1 for i in range(len(a)) if a[i] == 1 or b[i] == 1])
    return intersection / float(union)


input_ref_join = input_data.cartesian(reference_data)
input_ref_join = input_ref_join.map(lambda x: (x[0][0], x[1][0], jaccard_distance(x[0][1], x[1][1])))

closest_ref = input_ref_join.groupBy(lambda x: x[0]).map(lambda x: (x[0], max(x[1], key=lambda y: y[2])[1]))


for element in sorted(closest_ref.collect()):
    print(element)

('1', 'R1')
('2', 'R1')
('3', 'R1')
('4', 'R2')


In [ ]:
#Question no 3
input_data = sc.textFile("InputAuthor.txt")

# Extract authors and their coauthors
authors = input_data.map(lambda line: (line.split("->")[0].strip(), line.split("->")[1].strip().split(", ")))

# Find common coauthors
common_coauthors = authors.cartesian(authors).filter(lambda pair: pair[0][0] < pair[1][0])
cmn_coauthors = common_coauthors.flatMap(lambda pair: [((pair[0][0], pair[1][0]), coauthor) for coauthor in pair[0][1] if coauthor in pair[1][1]])

# Group coauthors 
grouping_coauthors = cmn_coauthors.groupByKey()

sorted_authors = grouping_coauthors.mapValues(lambda coauthors: (len(coauthors), sorted(coauthors)))
coauthors = sorted_authors.filter(lambda pair: pair[1][0] > 0)

# Print the results
for (author1, author2), (count, common_coauthors) in coauthors.collect():
    print(f"{author1}, {author2} -> ({count}) {', '.join(common_coauthors)}")

# Stop the Spark context
sc.stop()


B. Cao, C. Rego -> (1) Y. Lu
B. Cao, K. Kiani -> (1) Y. Lu
C. Rego, K. Kiani -> (2) B. Cao, Y. Lu
B. Cao, Y. Lu -> (3) C. Rego, F. Glover, K. Kiani
C. Rego, Y. Lu -> (1) B. Cao
B. Cao, F. Glover -> (1) Y. Lu
C. Rego, F. Glover -> (2) B. Cao, Y. Lu
B. Cao, B. Hosseini -> (1) K. Kiani
K. Kiani, Y. Lu -> (1) B. Cao
F. Glover, Y. Lu -> (1) B. Cao
B. Hosseini, Y. Lu -> (1) K. Kiani
F. Glover, K. Kiani -> (2) B. Cao, Y. Lu
